<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/analise_exploratoria_pacientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação
Este notebook foi desenvolvido como parte do projeto Monitor Rosa, um conjunto de ferramentas, softwares e outros artefatos cujo principal objetivo é melhorar o cenário de diagnósticos de câncer de mama no Brasil, idealmente melhorando diminuindo a proporção de diagnósticos tardios/diagnósticos precoces.

O arquivo utilizado nesta análise exploratória foi gerado a partir [deste notebook](https://github.com/heber-augusto/sus-kpis-analysis/blob/main/sia/indicadores_monitor_rosa_sia_pa.ipynb), que é um parte do [repositório de análises e criação de kpis](https://github.com/heber-augusto/sus-kpis-analysis).

# Dataset de Pacientes (1 linha por paciente)

Colunas:

 - Chave da paciente (cns_encrypted)
 - Custo total do tratamento
 - Estadiamento inicial
 - Estadiamento final
 - Indicação de óbito
 - Localização de moradia

## Premissas consideradas:

 - Tipos de arquivos a serem utilizados: AQ e AR
 - chave do paciente:
  - AQ: coluna AP_CNSPCN
  - AR: coluna AP_CNSPCN  
 - custo total do tratamento: será uma estimativa considerando apenas radioterapia e quimioterapia, calculado somando os valores dos procedimentos (presentes em AR e AQ), para cada chave de paciente. Sendo:
   - custos em AQ: soma de AP_VL_AP;
   - custos em AR: soma de AP_VL_AP.
 - Estadiamento inicial: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais antigo de radioterapia ou quimioterapia, de um determinado paciente;
 - Estadiamento final: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais recente de radioterapia ou quimioterapia, de um determinado paciente;
 - Localização de moradia: utilizar coluna AP_MUNPCN (presentes em AR e AQ). Talvez criar dois campos, AP_MUNPCN presente no registro mais antigo e AP_MUNPCN presente no registro mais novo;
 - Indicação de óbito: valor máximo do campo AP_OBITO, presente em AQ e AR (0: sem indicação e 1:com indicação de óbito) O valor 1 indicará óbito.

# Instalação de bibliotecas

## Bibliotecas utilizadas na análise exploratória

In [2]:
import pandas as pd
import numpy as np 

In [3]:
pacientes = pd.read_parquet('https://github.com/heber-augusto/sus-kpis-analysis/blob/a596c6f4f121c1ba3ae7b998143525657fa19837/sia/pacientes.parquet.gzip?raw=true')
pacientes.head()

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
0,898 0000 8725 6,201107,201108,2,2,2,3928.00,355220,355220,0
1,8980500671835 8,200903,200905,2,2,2,239.25,354890,354890,0
2,{{}{{{{{|,201004,201006,3,3,3,239.25,352900,352900,0
3,{}{|}{{|,200910,201105,1,1,1,4283.69,353660,353660,0
4,{}{~~~||{{{~,201102,201104,3,3,3,2400.00,510030,510030,0


In [9]:
pacientes.sort_values(by='data_primeiro_estadiamento')

,paciente,data_primeiro_estadiamento,data_ultimo_estadiamento,primeiro_estadiamento,maior_estadiamento,ultimo_estadiamento,custo_total,primeiro_municipio,ultimo_municipio,indicacao_obito
132357,{{|{{{{}~,200801,200802,1,1,1,159.50,355030,355030,0
119116,{{{|}{,200801,200807,2,4,2,7423.24,350170,350170,0
119117,{{{|},200801,200912,1,1,1,1914.00,350320,350320,0
119118,{{{|}},200801,201601,2,3,2,26833.50,355270,355370,0
119119,{{{|~{{~,200801,201502,2,3,2,24723.00,352930,352930,0
...,...,...,...,...,...,...,...,...,...,...
69340,{{{~}|}|},202202,202202,2,2,2,1400.00,354150,354150,0
41528,{||{~{{,202202,202202,1,1,1,79.75,350800,350800,0
102691,{|{|},202202,202202,2,2,2,1400.00,355030,355030,0
9227,}{{~}~||{{{,202202,202202,2,2,2,5904.00,354780,354780,0


## Transformação das colunas de data/hora

In [11]:
pacientes['data_primeiro_estadiamento_dt'] = pd.to_datetime(
    pacientes['data_primeiro_estadiamento'],
    format='%Y%m')

pacientes['data_ultimo_estadiamento_dt'] = pd.to_datetime(
    pacientes['data_ultimo_estadiamento'],
    format='%Y%m')